In [1]:
%load_ext autoreload
%load_ext autotime

time: 313 µs (started: 2023-03-16 14:10:45 -07:00)


In [2]:
import sys
sys.path.append("../")
import warnings
warnings.filterwarnings('ignore')
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

time: 1.26 s (started: 2023-03-16 14:10:45 -07:00)


In [3]:
import dask.dataframe as dd
import numpy as np
import pandas as pd


time: 514 ms (started: 2023-03-16 14:10:46 -07:00)


In [4]:
%autoreload 2

from wisio.recorder import RecorderAnalyzer

log_dir = "/p/vast1/iopp/recorder_app_logs/hacc/nodes-32/workflow-0/_parquet"

recorder_analyzer = RecorderAnalyzer(
    cluster_manager_args=dict(
        force_local=True,
        cluster_settings=dict(
            cores=4,
            dashboard_port=3656,
            local_directory="/var/tmp/dask-recorder",
            log_file="%J.log",
            worker_queue="pdebug"
        )
    ),
    working_dir='.recorder',
    debug=True
) 

recorder_analyzer

time: 2.1 s (started: 2023-03-16 14:10:47 -07:00)


In [5]:
[client.dashboard_link for client in recorder_analyzer.cluster_manager.clients]

['http://192.168.135.144:3656/status']

time: 3.46 ms (started: 2023-03-16 14:10:49 -07:00)


In [6]:
%autoreload 2

res = recorder_analyzer.analyze_parquet(log_dir=log_dir)

time: 4.9 s (started: 2023-03-16 14:10:49 -07:00)


In [7]:
views, bottlenecks = res


time: 657 µs (started: 2023-03-16 14:10:54 -07:00)


In [9]:
from wisio.rules import Rule
from wisio._recorder.rules import RecorderRuleEngine

rules = dict(
    file_name=[Rule.METADATA_ACCESS_ISSUE, Rule.SMALL_IO_ACCESS],
    proc_name=[Rule.METADATA_ACCESS_ISSUE],
    trange=[],
)

rule_engine = RecorderRuleEngine(rules=rules)

rule_engine.process_bottlenecks(bottlenecks=bottlenecks)


((<Rule.METADATA_ACCESS_ISSUE: 21>,
  ('file_name',),
  {'/p/gpfs1/iopp/temp/hacc_dir/test-Part00000000-of-00001280.data': RuleResult(rule=<Rule.METADATA_ACCESS_ISSUE: 21>, description="'/p/gpfs1/iopp/temp/hacc_dir/test-Part00000000-of-00001280.data' is accessed by 1 process(es) during the 13, 15, 19, 23, and 30th second(s) and spent 59.44% (3.86 seconds) of its I/O time on the 'open' operation(s).", reasons=[RuleReason(description='Metadata time is 59.44% (3.86 seconds) of I/O time', value=0.5944383099441845)]),
   '/p/gpfs1/iopp/temp/hacc_dir/test-Part00000001-of-00001280.data': RuleResult(rule=<Rule.METADATA_ACCESS_ISSUE: 21>, description="'/p/gpfs1/iopp/temp/hacc_dir/test-Part00000001-of-00001280.data' is accessed by 1 process(es) during the 13, 15, 19, 23, and 30th second(s) and spent 60.51% (3.84 seconds) of its I/O time on the 'open' operation(s).", reasons=[RuleReason(description='Metadata time is 60.52% (3.84 seconds) of I/O time', value=0.6052051539631613)]),
   '/p/gpfs1/iop

time: 16.5 s (started: 2023-03-16 14:12:25 -07:00)


In [11]:
for view_key, bottleneck in bottlenecks.items():
    for th in bottleneck.keys():
        print(view_key, th, len(bottleneck[th]))

('trange',) 0.001 27
('trange',) 0.010 27
('trange',) 0.100 19
('trange',) 0.250 10
('trange',) 0.500 7
('trange',) 0.750 3
('file_name',) 0.001 1280
('file_name',) 0.010 1280
('file_name',) 0.100 1208
('file_name',) 0.250 565
('file_name',) 0.500 252
('file_name',) 0.750 53
('proc_name',) 0.001 1280
('proc_name',) 0.010 1280
('proc_name',) 0.100 1208
('proc_name',) 0.250 565
('proc_name',) 0.500 252
('proc_name',) 0.750 53
('trange', 'file_name') 0.001 1280
('trange', 'file_name') 0.010 1280
('trange', 'file_name') 0.100 1208
('trange', 'file_name') 0.250 565
('trange', 'file_name') 0.500 252
('trange', 'file_name') 0.750 53
('trange', 'proc_name') 0.001 1280
('trange', 'proc_name') 0.010 1280
('trange', 'proc_name') 0.100 1208
('trange', 'proc_name') 0.250 565
('trange', 'proc_name') 0.500 252
('trange', 'proc_name') 0.750 53
('file_name', 'trange') 0.001 27
('file_name', 'trange') 0.010 27
('file_name', 'trange') 0.100 19
('file_name', 'trange') 0.250 10
('file_name', 'trange') 0.50

In [8]:
!rm -rf {log_dir}/bottlenecks

time: 394 ms (started: 2023-03-12 17:34:12 -07:00)


In [9]:
recorder_analyzer.save_bottlenecks(log_dir=log_dir, bottlenecks=bottlenecks)


time: 1min 29s (started: 2023-03-12 17:34:12 -07:00)


In [10]:
!chmod g+r {log_dir}/bottlenecks/*
!ls -lh {log_dir}/bottlenecks

total 313M
-rw-r--r-- 1 izzet iopp 18M Mar 12 17:34 file_name.json
-rw-r--r-- 1 izzet iopp 25M Mar 12 17:34 file_name_proc_name.json
-rw-r--r-- 1 izzet iopp 21M Mar 12 17:35 file_name_proc_name_trange.json
-rw-r--r-- 1 izzet iopp 21M Mar 12 17:34 file_name_trange.json
-rw-r--r-- 1 izzet iopp 25M Mar 12 17:35 file_name_trange_proc_name.json
-rw-r--r-- 1 izzet iopp 25M Mar 12 17:34 proc_name.json
-rw-r--r-- 1 izzet iopp 18M Mar 12 17:35 proc_name_file_name.json
-rw-r--r-- 1 izzet iopp 21M Mar 12 17:35 proc_name_file_name_trange.json
-rw-r--r-- 1 izzet iopp 21M Mar 12 17:35 proc_name_trange.json
-rw-r--r-- 1 izzet iopp 18M Mar 12 17:35 proc_name_trange_file_name.json
-rw-r--r-- 1 izzet iopp 21M Mar 12 17:34 trange.json
-rw-r--r-- 1 izzet iopp 18M Mar 12 17:34 trange_file_name.json
-rw-r--r-- 1 izzet iopp 25M Mar 12 17:35 trange_file_name_proc_name.json
-rw-r--r-- 1 izzet iopp 25M Mar 12 17:34 trange_proc_name.json
-rw-r--r-- 1 izzet iopp 18M Mar 12 17:35 trange_proc_name_file_name.json
ti